# Demo Notebook 
## for intercomparing AVISO maps geostrophic currents with across-track and along-track SWOT currents

In [ ]:
import xarray as xr

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
import sys
sys.path.append('..')
from src.mod_calc import *
from src.mod_plot import *

## Parameters

In [ ]:
params = {
    'dx' : 2000,                              # SWOT swath x-grid step in meters
    'dy' : 2000,                              # SWOT swath y-grid step in meters
    'stencil_npt' : 11, #9                    # stencil width: number of point for discretization
    'filtrage_m' : 300000,                    # lenght scale (in meters) equatorial filtering
    'lat_max_equator' : 10.,                  # maximum latitude (in degree) to consider for equatorial band processing (Recomm: not less than 10°)
    'maxval_current' : 10.                    # (absolute) maximum value of surface currents in meter / second
}

## Open SWOT swath

In [ ]:
filename = '../data/swot/aviso/SWOT_L2_LR_SSH_Expert_001_230_20160909T042119_20160909T051245_DG10_01.nc'
ds_descending = xr.open_dataset(filename)
filename = '../data/swot/aviso/SWOT_L2_LR_SSH_Expert_001_231_20160909T051246_20160909T060412_DG10_01.nc'
ds_ascending = xr.open_dataset(filename)

In [ ]:
ds_aviso_maps = xr.open_dataset('../data/aviso/dt_global_allsat_20160909.nc')

## Discretization scheme

#### First derivative discretisation scheme 

In [ ]:
show_discretisation_scheme(params['stencil_npt'], deriv=1)

#### Second derivative discretisation scheme 

In [ ]:
show_discretisation_scheme(params['stencil_npt'], deriv=2)

## Compute geostrophic currents

### Descending track

In [ ]:
u_final_descending, v_final_descending = compute_karin_geos_current(ds_descending.rename({'simulated_true_ssha_karin':'simulated_true_ssh_karin'}), params)

In [ ]:
u_zonal_final_descending, v_meridional_final_descending = convert_to_zonal_meridional(u_final_descending, v_final_descending)

In [ ]:
(u_final_descending.hvplot.quadmesh(x='longitude', 
                                    y='latitude', 
                                    clim=(-1., 1), 
                                    cmap='coolwarm', 
                                    datashade=True,
                                    shared_axes=True, 
                                    xlim=(136, 143), 
                                    ylim=(10, 40),
                                    title='Computed across-track geos. current') + u_zonal_final_descending.hvplot.quadmesh(x='longitude', 
                                                                                                                      y='latitude', 
                                                                                                                      clim=(-1., 1), 
                                                                                                                      cmap='coolwarm', 
                                                                                                                      datashade=True,
                                                                                                                        xlim=(136, 143), 
                                                                                                                      ylim=(10, 40),
                                                                                                                      title='SWOT Zonal current') + ds_aviso_maps.ugos.hvplot.quadmesh(x='longitude', 
                                                                                                                      y='latitude', 
                                                                                                                      clim=(-1., 1), 
                                                                                                                      cmap='coolwarm', 
                                                                                                                      datashade=True,
                                                                                                                      shared_axes=True, 
                                                                                                                      xlim=(136, 143), 
                                                                                                                      ylim=(10, 40),
                                                                                                                      title='AVISO maps Zonal current')
).cols(1)

In [ ]:
(v_final_descending.hvplot.quadmesh(x='longitude', 
                                    y='latitude', 
                                    clim=(-1., 1), 
                                    cmap='coolwarm', 
                                    datashade=True,
                                    shared_axes=True, 
                                    xlim=(136, 143), 
                                    ylim=(10, 40),
                                    title='Computed along-track geos. current') + v_meridional_final_descending.hvplot.quadmesh(x='longitude', 
                                                                                                                      y='latitude', 
                                                                                                                      clim=(-1., 1), 
                                                                                                                      cmap='coolwarm', 
                                                                                                                      datashade=True,
                                                                                                                        xlim=(136, 143), 
                                                                                                                      ylim=(10, 40),
                                                                                                                      title='SWOT Meridional current') + ds_aviso_maps.vgos.hvplot.quadmesh(x='longitude', 
                                                                                                                     y='latitude', 
                                                                                                                     clim=(-1., 1), 
                                                                                                                     cmap='coolwarm', 
                                                                                                                     datashade=True, 
                                                                                                                     shared_axes=True, 
                                                                                                                     xlim=(136, 143), 
                                                                                                                     ylim=(10, 40),
                                                                                                                     title='AVISO maps Meridional current')
).cols(1)

### Ascending track

In [ ]:
u_final_ascending, v_final_ascending = compute_karin_geos_current(ds_ascending.rename({'simulated_true_ssha_karin':'simulated_true_ssh_karin'}), params)

In [ ]:
u_zonal_final_ascending, v_meridional_final_ascending = convert_to_zonal_meridional(u_final_ascending, v_final_ascending)

In [ ]:
def selection(ds, lon_min, mon_max, lat_min, lat_max):
    ds_tmp = ds.where((ds.longitude >= lon_min) & (ds.longitude <= lon_max), drop=True)
    ds_tmp = ds_tmp.where((ds_tmp.latitude >= lat_min) & (ds_tmp.latitude <= lat_max), drop=True)
    return ds_tmp

In [ ]:
lon_min = 315
lon_max = 320
lat_min = 30
lat_max = 40

In [ ]:
(selection(u_final_ascending, lon_min, lon_max, lat_min, lat_max).hvplot.quadmesh(x='longitude', 
                                   y='latitude', 
                                   clim=(-0.5, 0.5), 
                                   cmap='coolwarm', 
                                   #datashade=True,
                                   shared_axes=True, 
                                   xlim=(315, 320), 
                                   ylim=(30, 40),
                                   title='Computed across-track geos. current') + selection(u_zonal_final_ascending, lon_min, lon_max, lat_min, lat_max).hvplot.quadmesh(x='longitude', 
                                                                                                                     y='latitude', 
                                                                                                                     clim=(-0.5, 0.5), 
                                                                                                                     cmap='coolwarm', 
                                                                                                                     #datashade=True,
                                                                                                                     shared_axes=True, 
                                                                                                                     xlim=(315, 320), 
                                                                                                                     ylim=(30, 40),
                                                                                                                     title='SWOT Zonal current') + selection(ds_aviso_maps, lon_min, lon_max, lat_min, lat_max).ugos.hvplot.quadmesh(x='longitude', 
                                                                                                                     y='latitude', 
                                                                                                                     clim=(-0.5, 0.5), 
                                                                                                                     cmap='coolwarm', 
                                                                                                                     #datashade=True,
                                                                                                                     shared_axes=True, 
                                                                                                                     xlim=(315, 320), 
                                                                                                                     ylim=(30, 40),
                                                                                                                     title='AVISO maps Zonal current')
).cols(1)

In [ ]:
(v_final_ascending.hvplot.quadmesh(x='longitude', 
                                   y='latitude', 
                                   clim=(-1., 1), 
                                   cmap='coolwarm', 
                                   datashade=True,
                                   shared_axes=True, 
                                   xlim=(315, 320), 
                                   ylim=(30, 40),
                                   title='Computed along-track geos. current') + v_meridional_final_ascending.hvplot.quadmesh(x='longitude', 
                                   y='latitude', 
                                   clim=(-1., 1), 
                                   cmap='coolwarm', 
                                   datashade=True,
                                   shared_axes=True, 
                                   xlim=(315, 320), 
                                   ylim=(30, 40),
                                   title='SWOT Meridional current') + ds_aviso_maps.vgos.hvplot.quadmesh(x='longitude', 
                                                                                                                    y='latitude', 
                                                                                                                    clim=(-1., 1), 
                                                                                                                    cmap='coolwarm', 
                                                                                                                    datashade=True,
                                                                                                                    shared_axes=True, 
                                                                                                                    xlim=(315, 320), 
                                                                                                                    ylim=(30, 40),
                                                                                                                    title='AVISO maps Meridional current')
).cols(1)